<a href="https://colab.research.google.com/github/farsim-hossain/python-projects/blob/main/Loading_and_Saving_A_Model_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading and Saving Model 

In this exercise, we will learn how to load and save models in TF 

There is a dependency for saving a model in HDFS format. That is called pyyaml h5py

In [2]:
import os
import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.4.1


## Get an example Dataset 

To demonstrate how er load and train weights, we will use MNIST dataset. 

In [4]:
(train_images, train_labels), (test_images,test_labels) = tf.keras.datasets.mnist.load_data()
train_labels = train_labels[:1000] # just getting 1000 samples to make the process shorter
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28*28)/255.0 
test_images = test_images[:1000].reshape(-1, 28*28)/255.0



## Defining a model

In [6]:
def create_model():
  model = tf.keras.models.Sequential([
                                      keras.layers.Dense(512, activation='relu', input_shape = (784,)),
                                      keras.layers.Dropout(0.2),
                                      keras.layers.Dense(10)
  ])

  model.compile(
      optimizer = 'adam',
      loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics = [tf.metrics.SparseCategoricalAccuracy()]
  )

  return model

model = create_model()

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Save checkpoints during training

You can use a trained model without having to retrain it, or pick-up training where you left off in case the training process was interrupted. The tf.keras.callbacks.ModelCheckpoint callback allows you to continually save the model both during and at the end of training.



In [7]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Creating a callback that saves the model's weights 

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose= 1)

# train the model with new callback 

model.fit(
    train_images,
    train_labels,
    epochs = 10,
    validation_data = (test_images, test_labels),
    callbacks = [cp_callback]
)

Epoch 1/10
32/32 [==============================] - 1s 16ms/step - loss: 1.6042 - sparse_categorical_accuracy: 0.4845 - val_loss: 0.7348 - val_sparse_categorical_accuracy: 0.7760

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 8ms/step - loss: 0.4456 - sparse_categorical_accuracy: 0.8796 - val_loss: 0.5392 - val_sparse_categorical_accuracy: 0.8280

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 8ms/step - loss: 0.2970 - sparse_categorical_accuracy: 0.9210 - val_loss: 0.4879 - val_sparse_categorical_accuracy: 0.8520

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 8ms/step - loss: 0.2160 - sparse_categorical_accuracy: 0.9404 - val_loss: 0.4736 - val_sparse_categorical_accuracy: 0.8530

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 9ms/step - loss: 0.1658 - sparse_cat

Now we can see a checkpoint location and what is inside it.

In [8]:
os.listdir(checkpoint_dir)

['cp.ckpt.data-00000-of-00001', 'checkpoint', 'cp.ckpt.index']

## Recalling weights from another model 
As long as two models share the same architecture you can share weights between them. So, when restoring a model from weights-only, create a model with the same architecture as the original model and then set its weights.

Lets check with a new untrained model which might give a 10% accuracy, doesnt matter. We will update the model with our trained model weights.

In [9]:
model = create_model()

#evaluate the model 

loss, acc = model.evaluate(test_images, test_labels, verbose = 2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))



32/32 - 0s - loss: 2.3111 - sparse_categorical_accuracy: 0.1670
Untrained model, accuracy: 16.70%


## Loading the weights and reevaluate the model 


In [10]:
model.load_weights(checkpoint_path)

loss,acc = model.evaluate(test_images, test_labels, verbose = 2)
print("Restored model, accuracy:{:5.2f}".format(100*acc))

32/32 - 0s - loss: 0.3998 - sparse_categorical_accuracy: 0.8700
Restored model, accuracy:87.00


## Checkpoints saving options with different names

The callback provides several options to provide unique names for checkpoints and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every five epochs:

In [12]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

#new callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size) # the important takeaway

model = create_model()

# Save the weights using the `checkpoint_path` format

model.save_weights(checkpoint_path.format(epoch=0))


# train with new callback 

model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)




Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


## Checking the list of checkpoints again 

In [13]:
os.listdir(checkpoint_dir)

['cp-0050.ckpt.index',
 'cp-0010.ckpt.index',
 'cp-0030.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'checkpoint',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.index',
 'cp-0015.ckpt.index',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index',
 'cp-0035.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0005.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.data-00000-of-00001']

## Getting the latest checkpoint

In [14]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

## To test, reset the model and loading the latest checkpoint 


In [15]:
model = create_model()

model.load_weights(latest)

loss ,acc = model.evaluate(test_images, test_labels, verbose =2)
print("Restored model, accuracy : {:5.2f}%".format(100*acc))



32/32 - 0s - loss: 0.4799 - sparse_categorical_accuracy: 0.8790
Restored model, accuracy : 87.90%


## Manually Save Weights 

Manually saving weights with the Model.save_weights method. By default, tf.keras—and save_weights in particular—uses the TensorFlow checkpoint format with a .ckpt extension (saving in HDF5 with a .h5 extension is covered in the Save and serialize models guide)

In [16]:
#save weights 

model.save_weights('./checkpoints/my_checkpoint')

#Create a new model instance 
model = create_model()

#restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# evaluate the model 
loss,acc = model.evaluate(test_images, test_labels, verbose =2)
print("Restored model, accuracy:{:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4799 - sparse_categorical_accuracy: 0.8790
Restored model, accuracy:87.90%


## Save the entire model 

Call model.save to save a model's architecture, weights, and training configuration in a single file/folder. This allows you to export a model so it can be used without access to the original Python code*. Since the optimizer-state is recovered, you can resume training from exactly where you left off.

An entire model can be saved in two different file formats (SavedModel and HDF5). The TensorFlow SavedModel format is the default file format in TF2.x. However, models can be saved in HDF5 format. More details on saving entire models in the two file formats is described below.

Saving a fully-functional model is very useful—you can load them in TensorFlow.js (Saved Model, HDF5) and then train and run them in web browsers, or convert them to run on mobile devices using TensorFlow Lite (Saved Model, HDF5)

*Custom objects (e.g. subclassed models or layers) require special attention when saving and loading. See the Saving custom objects section below

## SavedModel format 

The SavedModel format is another way to serialize models. Models saved in this format can be restored using tf.keras.models.load_model and are compatible with TensorFlow Serving. The SavedModel guide goes into detail about how to serve/inspect the SavedModel. The section below illustrates the steps to save and restore the model.

In [17]:
## example create and train a new model instance 

model = create_model()
model.fit(train_images, train_labels, epochs= 5)

## Save the entire model as a saved model 

!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 1s 5ms/step - loss: 1.6237 - sparse_categorical_accuracy: 0.4784
Epoch 2/5
32/32 [==============================] - 0s 6ms/step - loss: 0.4476 - sparse_categorical_accuracy: 0.8769
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2833 - sparse_categorical_accuracy: 0.9236
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2274 - sparse_categorical_accuracy: 0.9328
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1901 - sparse_categorical_accuracy: 0.9629
INFO:tensorflow:Assets written to: saved_model/my_model/assets


The SavedModel format is a directory containing a protobuf binary and a Tensorflow checkpoint. Inspect the saved model directory: 


In [18]:
!ls saved_model

my_model


In [19]:
!ls saved_model/my_model

assets	saved_model.pb	variables


## Lets reload a fresh keras model and load opur model 

In [20]:
new_model = tf.keras.models.load_model('saved_model/my_model')

new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


The restored model is compiled with the same arguments as the original model. Try running evaluate and predict with the loaded model:

In [21]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose =2)

print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4523 - sparse_categorical_accuracy: 0.8420
Restored model, accuracy: 84.20%
(1000, 10)


## HDF5 Format

There is another way to save the model

In [22]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.6517 - sparse_categorical_accuracy: 0.4784
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.5075 - sparse_categorical_accuracy: 0.8718
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.3233 - sparse_categorical_accuracy: 0.9182
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2289 - sparse_categorical_accuracy: 0.9480
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1661 - sparse_categorical_accuracy: 0.9583


Lets recrate the model from this file


In [23]:
new_model = tf.keras.models.load_model('my_model.h5')

new_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [24]:
# accuracy check 

loss, acc = new_model.evaluate(test_images, test_labels, verbose = 2)

print("Restored model , accuracy: {:5.2f}".format(100*acc))

32/32 - 0s - loss: 0.4209 - sparse_categorical_accuracy: 0.8720
Restored model , accuracy: 87.20


Keras saves models by inspecting their architectures. This technique saves everything:

The weight values
The model's architecture
The model's training configuration (what you pass to the .compile() method)
The optimizer and its state, if any (this enables you to restart training where you left off)
Keras is not able to save the v1.x optimizers (from tf.compat.v1.train) since they aren't compatible with checkpoints. For v1.x optimizers, you need to re-compile the model after loading—losing the state of the optimizer.